# preparing the Data
this process consists of a few major stages:
* merging the data from the same trusts every month
by picking out the C_999 function code, it returns the sum for each type of appointment
running groupby sums each appointment together leaving the total number of appointments for a trust
```py
data = data.loc[data['Treatment Function Code'] == "C_999"]
hospital_grouped = data.groupby('Provider Org Name')
```

* converting column names to be human and machine-readable
when grouping the data, strings will be appended. we prevent this with a similar check

* converting time stamps

the data per week was stored in the form {before} to {after} weeks SUM 1
by taking just {after} we are left with an easier to read column name

```py
# getting the columns
timecols = [col for col in hospital_summed if "SUM 1" in col]
# finding the correct week
week_values = [int(col.split(" ")[1]) for col in timecols]
# renaming the data
renamed = {col: week_values[i] for i, col in enumerate(timecols)}
out = hospital_summed.rename(columns=renamed)
```
* merging the monthly data

In [1]:
import pandas as pd
import plotly

In [2]:


months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
years = ["2021", "2022", "2023"]
df = None

for year in years:
    print(f"loading {year}")
    for month in months:
        print(f"loading {month}")
        if year == "2019" and month not in [""]:continue
        if year == "2023" and month == "april":
            break
        data = pd.read_csv(f"data/apptx/{year}/{month}.csv")
        data = data.loc[data['Treatment Function Code'] == "C_999"]
        hospital_grouped = data.groupby('Provider Org Name')

        agg_functions = {}
        for col in data.columns:
            if data[col].dtype in [int, float]:
                agg_functions[col] = 'sum'
            else:
                agg_functions[col] = 'first'

        hospital_summed = hospital_grouped.agg(agg_functions)
        timecols = [col for col in hospital_summed if "SUM 1" in col]
        week_values = [int(col.split(" ")[1]) for col in timecols]
        renamed = {col: week_values[i] for i, col in enumerate(timecols)}
        out = hospital_summed.rename(columns=renamed)


        out["Period"] = out["Period"].str.slice(start=4)
        out = out.drop(columns=["Total","Patients with unknown clock start date", "Total All"])

        out.reset_index(drop=True, inplace=True)  # Reset index without adding existing column

        print(f"finished {year}.{month}")
        if df is None:
            df = out
        else:
            df = pd.concat((df,out), axis=0)
# df.to_csv("data/all_hospital.csv")
df.head()


loading 2021
loading january
finished 2021.january
loading february
finished 2021.february
loading march
finished 2021.march
loading april
finished 2021.april
loading may
finished 2021.may
loading june
finished 2021.june
loading july
finished 2021.july
loading august
finished 2021.august
loading september
finished 2021.september
loading october
finished 2021.october
loading november
finished 2021.november
loading december
finished 2021.december
loading 2022
loading january
finished 2022.january
loading february
finished 2022.february
loading march
finished 2022.march
loading april
finished 2022.april
loading may
finished 2022.may
loading june
finished 2022.june
loading july
finished 2022.july
loading august
finished 2022.august
loading september
finished 2022.september
loading october
finished 2022.october
loading november
finished 2022.november
loading december
finished 2022.december
loading 2023
loading january
finished 2023.january
loading february
finished 2023.february
loading mar

,Period,Provider Parent Org Code,Provider Parent Name,Provider Org Code,Provider Org Name,Commissioner Parent Org Code,Commissioner Parent Name,Commissioner Org Code,Commissioner Org Name,RTT Part Type,...,95,96,97,98,99,100,101,102,103,104
0,JANUARY-2021,QWO,WEST YORKSHIRE AND HARROGATE HEALTH & CARE PAR...,RCF,AIREDALE NHS FOUNDATION TRUST,QE1,HEALTHIER LANCASHIRE AND SOUTH CUMBRIA,X24,NHS ENGLAND,Part_1A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JANUARY-2021,QYG,CHESHIRE AND MERSEYSIDE STP,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,QE1,HEALTHIER LANCASHIRE AND SOUTH CUMBRIA,NONC,NHS ENGLAND,Part_1A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JANUARY-2021,QUE,CAMBRIDGESHIRE AND PETERBOROUGH STP,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,QUE,CAMBRIDGESHIRE AND PETERBOROUGH STP,06H,NHS CAMBRIDGESHIRE AND PETERBOROUGH CCG,Part_1A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JANUARY-2021,QXU,SURREY HEARTLANDS HEALTH & CARE PARTNERSHIP (STP),RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,QE1,HEALTHIER LANCASHIRE AND SOUTH CUMBRIA,13Q,NATIONAL COMMISSIONING HUB 1,Part_1B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JANUARY-2021,QXU,SURREY HEARTLANDS HEALTH & CARE PARTNERSHIP (STP),NVC01,ASHTEAD HOSPITAL,QE1,HEALTHIER LANCASHIRE AND SOUTH CUMBRIA,01A,NHS EAST LANCASHIRE CCG,Part_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
dframe = df.drop(columns=["Provider Parent Org Code","Provider Parent Name","Commissioner Parent Org Code","Commissioner Parent Name","Commissioner Org Name","Commissioner Org Code","Treatment Function Name","Treatment Function Code", "RTT Part Description", "RTT Part Type"])
dframe

,Period,Provider Org Code,Provider Org Name,0,1,2,3,4,5,6,...,95,96,97,98,99,100,101,102,103,104
0,JANUARY-2021,RCF,AIREDALE NHS FOUNDATION TRUST,1082.0,1028.0,669.0,724.0,381.0,553.0,525.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JANUARY-2021,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,487.0,376.0,347.0,483.0,314.0,475.0,688.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JANUARY-2021,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,26.0,104.0,138.0,173.0,66.0,55.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JANUARY-2021,RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,1932.0,2368.0,2365.0,2388.0,1478.0,1770.0,2116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JANUARY-2021,NVC01,ASHTEAD HOSPITAL,45.0,47.0,50.0,75.0,28.0,34.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,MARCH-2023,RWP,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,4292.0,3703.0,2887.0,2814.0,2800.0,2654.0,2602.0,...,24.0,31.0,26.0,30.0,17.0,13.0,6.0,9.0,8.0,12.0
447,MARCH-2023,RRF,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",3537.0,3835.0,2717.0,2687.0,2566.0,2467.0,2334.0,...,5.0,4.0,0.0,1.0,2.0,2.0,2.0,0.0,0.0,7.0
448,MARCH-2023,RLQ,WYE VALLEY NHS TRUST,2090.0,1759.0,1556.0,1455.0,1286.0,1263.0,1239.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0
449,MARCH-2023,RA4,YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,1783.0,1190.0,1075.0,898.0,919.0,835.0,829.0,...,2.0,1.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,3.0


In [4]:
dframe['Total'] = dframe.iloc[:,3:].T.sum()
dframe

,Period,Provider Org Code,Provider Org Name,0,1,2,3,4,5,6,...,96,97,98,99,100,101,102,103,104,Total
0,JANUARY-2021,RCF,AIREDALE NHS FOUNDATION TRUST,1082.0,1028.0,669.0,724.0,381.0,553.0,525.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13274.0
1,JANUARY-2021,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,487.0,376.0,347.0,483.0,314.0,475.0,688.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13600.0
2,JANUARY-2021,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,26.0,104.0,138.0,173.0,66.0,55.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1579.0
3,JANUARY-2021,RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,1932.0,2368.0,2365.0,2388.0,1478.0,1770.0,2116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37077.0
4,JANUARY-2021,NVC01,ASHTEAD HOSPITAL,45.0,47.0,50.0,75.0,28.0,34.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1072.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,MARCH-2023,RWP,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,4292.0,3703.0,2887.0,2814.0,2800.0,2654.0,2602.0,...,31.0,26.0,30.0,17.0,13.0,6.0,9.0,8.0,12.0,88961.0
447,MARCH-2023,RRF,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",3537.0,3835.0,2717.0,2687.0,2566.0,2467.0,2334.0,...,4.0,0.0,1.0,2.0,2.0,2.0,0.0,0.0,7.0,66859.0
448,MARCH-2023,RLQ,WYE VALLEY NHS TRUST,2090.0,1759.0,1556.0,1455.0,1286.0,1263.0,1239.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,36290.0
449,MARCH-2023,RA4,YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,1783.0,1190.0,1075.0,898.0,919.0,835.0,829.0,...,1.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,3.0,22011.0


# we then save this data into an CSV (this can be exchanged for xlsx)

In [5]:
dframe.to_csv("data/all_hospital.csv",index=False)
dframe

,Period,Provider Org Code,Provider Org Name,0,1,2,3,4,5,6,...,96,97,98,99,100,101,102,103,104,Total
0,JANUARY-2021,RCF,AIREDALE NHS FOUNDATION TRUST,1082.0,1028.0,669.0,724.0,381.0,553.0,525.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13274.0
1,JANUARY-2021,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,487.0,376.0,347.0,483.0,314.0,475.0,688.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13600.0
2,JANUARY-2021,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,26.0,104.0,138.0,173.0,66.0,55.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1579.0
3,JANUARY-2021,RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,1932.0,2368.0,2365.0,2388.0,1478.0,1770.0,2116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37077.0
4,JANUARY-2021,NVC01,ASHTEAD HOSPITAL,45.0,47.0,50.0,75.0,28.0,34.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1072.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,MARCH-2023,RWP,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,4292.0,3703.0,2887.0,2814.0,2800.0,2654.0,2602.0,...,31.0,26.0,30.0,17.0,13.0,6.0,9.0,8.0,12.0,88961.0
447,MARCH-2023,RRF,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",3537.0,3835.0,2717.0,2687.0,2566.0,2467.0,2334.0,...,4.0,0.0,1.0,2.0,2.0,2.0,0.0,0.0,7.0,66859.0
448,MARCH-2023,RLQ,WYE VALLEY NHS TRUST,2090.0,1759.0,1556.0,1455.0,1286.0,1263.0,1239.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,36290.0
449,MARCH-2023,RA4,YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,1783.0,1190.0,1075.0,898.0,919.0,835.0,829.0,...,1.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,3.0,22011.0


# we then convert this into a percentage of the total to account for larger hospitals having more patients and therefore more people waiting

In [6]:

percentage = dframe.columns[3:-1]

percentage_new = dframe
percentage_new[percentage] = dframe[percentage].div(dframe.Total, axis=0)*100
percentage_new

,Period,Provider Org Code,Provider Org Name,0,1,2,3,4,5,6,...,96,97,98,99,100,101,102,103,104,Total
0,JANUARY-2021,RCF,AIREDALE NHS FOUNDATION TRUST,8.151273,7.744463,5.039928,5.454272,2.870273,4.166039,3.955100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13274.0
1,JANUARY-2021,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,3.580882,2.764706,2.551471,3.551471,2.308824,3.492647,5.058824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13600.0
2,JANUARY-2021,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,1.646612,6.586447,8.739709,10.956301,4.179861,3.483217,7.979734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1579.0
3,JANUARY-2021,RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,5.210778,6.386709,6.378617,6.440651,3.986299,4.773849,5.707042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37077.0
4,JANUARY-2021,NVC01,ASHTEAD HOSPITAL,4.197761,4.384328,4.664179,6.996269,2.611940,3.171642,5.970149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1072.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,MARCH-2023,RWP,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,4.824586,4.162498,3.245242,3.163184,3.147447,2.983330,2.924877,...,0.034847,0.029226,0.033723,0.019109,0.014613,0.006745,0.010117,0.008993,0.013489,88961.0
447,MARCH-2023,RRF,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",5.290238,5.735952,4.063776,4.018905,3.837928,3.689855,3.490929,...,0.005983,0.000000,0.001496,0.002991,0.002991,0.002991,0.000000,0.000000,0.010470,66859.0
448,MARCH-2023,RLQ,WYE VALLEY NHS TRUST,5.759162,4.847065,4.287683,4.009369,3.543676,3.480298,3.414164,...,0.000000,0.000000,0.002756,0.005511,0.000000,0.000000,0.000000,0.000000,0.005511,36290.0
449,MARCH-2023,RA4,YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,8.100495,5.406388,4.883922,4.079778,4.175185,3.793558,3.766299,...,0.004543,0.000000,0.009086,0.004543,0.000000,0.004543,0.004543,0.004543,0.013630,22011.0


In [7]:
percentage_new.to_csv("data/percentage_hospital.csv", index=False)